In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#import the CIFAR-10 dataset
(x_train,y_train),(x_test,y_test)=keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_test.shape)

170508288/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)
(10000, 1)


In [3]:
#https://keras.io/api/datasets/cifar10/
labels=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [4]:
#normalize pixel values
x_train=x_train/255.0
x_test=x_test/255.0

In [5]:
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [6]:
#final testing and training data
x_train1,x_test1,y_train1,y_test1=train_test_split(x_train,y_train,test_size=0.2,random_state=3)

In [7]:
print(x_train1.shape,y_train1.shape)

(40000, 32, 32, 3) (40000, 1)


In [8]:
#one hot encoding of train and test data
y_train1=keras.utils.to_categorical(y_train1,num_classes=10,dtype='uint8')
y_test1=keras.utils.to_categorical(y_test1,num_classes=10,dtype='uint8')

In [9]:
print(x_train1.shape,y_train1.shape)

(40000, 32, 32, 3) (40000, 10)


In [10]:
from keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,\
     Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add 
from keras import Sequential

In [11]:
#residual block with 2 layers
def add_residual_block(model,dimension):
    initial=model
    model = (Conv2D(dimension,kernel_size=(3,3),padding='same',activation='relu'))(model)
    model = (BatchNormalization())(model)
    model = (Conv2D(dimension,kernel_size=(3,3),padding='same',activation='relu'))(model)
    model = (BatchNormalization())(model)
    if(model.shape[3]!=initial.shape[3]):  #dimensionality mismatch between initial and final layer
        initial = (Conv2D(dimension,(1,1),padding='same',activation='relu'))(model)
    model=Add()([model,initial])
    return model

In [12]:
#size reducing residual block (fixed to halve size)
def add_residual_sizereduction(model,dimension):
    initial=model
    #reduce size
    model = ZeroPadding2D(padding=(1,1))(model)
    print("def ",model.shape,initial.shape)
    model = (Conv2D(dimension,kernel_size=(3,3),strides=(2,2),padding='valid',activation='relu'))(model)
    model = (BatchNormalization())(model)
    print("def ",model.shape,initial.shape)
    model = (Conv2D(dimension,kernel_size=(3,3),padding='same',activation='relu'))(model)
    model = (BatchNormalization())(model)
    print("def ",model.shape,initial.shape)
#     if(model.shape[3]!=initial.shape[3]):  #dimensionality mismatch between initial and final layer
#         initial = (Conv2D(dimension,kernel_size=(1,1),padding='same',activation='relu'))(model)
    initial = (Conv2D(dimension,kernel_size=(1,1),strides=(1,1),padding='valid',activation='relu'))(model)
    print("def ",model.shape,initial.shape)
    model=Add()([model,initial])
    return model

In [13]:
print(x_train1.shape)

(40000, 32, 32, 3)


In [14]:
# model=Sequential()
model1 = Input(shape=(x_train1.shape[1],x_train1.shape[2],x_train1.shape[3]))
model = (Conv2D(32,kernel_size=(3,3),padding='same',activation='relu'))(model1)
print(model.shape)
model = add_residual_block(model,32)
model = add_residual_block(model,32)
print(model.shape)
model = add_residual_sizereduction(model,64)
model = add_residual_block(model,64)
model = add_residual_block(model,64)
model = add_residual_sizereduction(model,128)
model = add_residual_block(model,128)
model = add_residual_block(model,128)
model = (Flatten())(model)
# model = (Dense(128,activation='relu'))(model)
model = (Dense(10,activation='softmax'))(model)
model = keras.Model(inputs=model1,outputs=model,name='Resnet')

2023-01-15 16:31:42.851602: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-15 16:31:42.943526: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-15 16:31:42.944387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-15 16:31:42.946063: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

(None, 32, 32, 32)
(None, 32, 32, 32)
def  (None, 34, 34, 32) (None, 32, 32, 32)
def  (None, 16, 16, 64) (None, 32, 32, 32)
def  (None, 16, 16, 64) (None, 32, 32, 32)
def  (None, 16, 16, 64) (None, 16, 16, 64)
def  (None, 18, 18, 64) (None, 16, 16, 64)
def  (None, 8, 8, 128) (None, 16, 16, 64)
def  (None, 8, 8, 128) (None, 16, 16, 64)
def  (None, 8, 8, 128) (None, 8, 8, 128)


In [15]:
model.summary()

Model: "Resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   9248        conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 32)   128         conv2d_1[0][0]                   
_____________________________________________________________________________________________

In [16]:
print(x_train1.shape)
print(y_train1.shape)

(40000, 32, 32, 3)
(40000, 10)


In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
resnet_train = model.fit(x_train1,y_train1,epochs=80)


2023-01-15 16:31:47.337024: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/80


2023-01-15 16:31:50.293142: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1250/1250 [==============================] - 24s 12ms/step - loss: 2.1272 - accuracy: 0.4057
Epoch 2/80
1250/1250 [==============================] - 15s 12ms/step - loss: 1.2499 - accuracy: 0.5823
Epoch 3/80
1250/1250 [==============================] - 15s 12ms/step - loss: 1.0090 - accuracy: 0.6635
Epoch 4/80
1250/1250 [==============================] - 15s 12ms/step - loss: 0.8638 - accuracy: 0.7094
Epoch 5/80
1250/1250 [==============================] - 14s 11ms/step - loss: 0.7510 - accuracy: 0.7466
Epoch 6/80
1250/1250 [==============================] - 15s 12ms/step - loss: 0.6452 - accuracy: 0.7828
Epoch 7/80
1250/1250 [==============================] - 14s 11ms/step - loss: 0.5429 - accuracy: 0.8181
Epoch 8/80
1250/1250 [==============================] - 15s 12ms/step - loss: 0.4441 - accuracy: 0.8492
Epoch 9/80
1250/1250 [==============================] - 14s 12ms/step - loss: 0.3577 - accuracy: 0.8785
Epoch 10/80
1250/1250 [==============================] - 15s 12ms/step - lo

KeyboardInterrupt: 

In [18]:
model.evaluate(x_test1,y_test1)

313/313 [==============================] - 2s 5ms/step - loss: 1.6463 - accuracy: 0.7621


[1.6462990045547485, 0.7620999813079834]